# Machine Learning Regressoin Model for Housing Prices tuned Using Optuna and CatBoostRegressor

## The following cell has the code for installing the libraries that are not commonly found in python installations. If they are already installed just delete the cell or comment it out.

In [ ]:
!pip install catboost --user
!pip install optuna --user

## The folllowing is the model for the dataset on Housing Prices and the prediction by using CatBoostRegressor. The train.csv dataset contains 1459 entries and the test.csv dataset contains 1459 entries to which the prediction is outputed into the sample_submission.csv. You can follow through to see the operations performed on the data and the model built for the prediction.
## The first cell contains all the neccesary libraries that will be used in the model and all the preprocessing activities.

In [ ]:
import catboost
import optuna
from scipy import stats
import datetime as dt
import time
import sklearn.metrics as metrics
import numpy as np
import pandas as pd
from catboost import Pool
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures as pf
from sklearn.model_selection import  train_test_split, GridSearchCV
from sklearn.metrics import *
from catboost import CatBoostRegressor
import math
from sklearn import tree, model_selection
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## The cell below is used to read the .csv files into pandas dataframes that we shall be working on throughout the entire notebook.

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sub = pd.read_csv("sample_submission.csv")

## The Cell below contains code to allow 100 outputs to be seen to allow me to see the features to discard and the the features to keep.  

In [ ]:
pd.set_option("display.max_rows", 100)

## The Cell below contains code to look for the null values in the train dataset and show them as percentages.

In [ ]:
train.isnull().mean()*100

## The Cell below contains code to drop the unusable columns in the train.csv dataset that have too many null values to be useful features.

In [ ]:
X = train.drop(["LotFrontage","Alley","FireplaceQu","PoolQC","Fence","MiscFeature","Id"], axis=1)

## The Cell below contains code to view all the unique values in each column. If a column has alot of unique values, they have to be continous data values or it cannot be useful as a feature and also if it has very few unique values then it has to be categorical data values or else it cannot be a useful feature.

In [ ]:
X.nunique().sort_values()

## The Cell below contains code to sort the columns according to their different data types so that I can get all the categorical data columns into the next cell.

In [ ]:
X.dtypes.sort_values()

## The Cell below contains code to change all the categorrical columns into integers so that I can work on them to find the P-value and also to use for separation from the rest of the dataset to be left with the continous data for correlation analysis using RegPlot.

In [ ]:
X = X.astype({'GarageFinish' : 'category' ,'Condition2' : 'category' ,'Condition1' : 'category',
'GarageQual' : 'category','GarageCond' : 'category','BsmtExposure' : 'category','Neighborhood' : 'category',
'LandSlope' : 'category','LotConfig' : 'category','Utilities' : 'category','LandContour' : 'category',
'LotShape' : 'category','Street' : 'category','MSZoning' : 'category','SaleType' : 'category',
'PavedDrive' : 'category','GarageType' : 'category','HouseStyle' : 'category','Functional' : 'category',
'BsmtFinType1' : 'category','BsmtQual' : 'category','BsmtFinType2' : 'category','Foundation' : 'category',
'ExterCond' : 'category','ExterQual' : 'category','SaleCondition' : 'category','HeatingQC' : 'category',
'CentralAir' : 'category','Electrical' : 'category','MasVnrType' : 'category','Exterior2nd' : 'category',
'Exterior1st' : 'category','RoofMatl' : 'category','RoofStyle' : 'category','KitchenQual' : 'category',
'BsmtCond' : 'category','BldgType' : 'category','Heating' : 'category'})
cat_columns = X.select_dtypes(['category']).columns
X[cat_columns] = X[cat_columns].apply(lambda x: x.cat.codes)
X.dtypes.sort_values()

## The Cell below contains code to perform the correlation analysis on the continous data columns and only use those that pass a certain threshhold seen in the if-else statement in the cell. Only the columns that pass this test will be used in the RegPlot section of the cell and plotted out.

In [ ]:
c_n_housing1 = X.columns.values.tolist()
unwanted_num=["SalePrice","GarageFinish","Condition2","Condition1","GarageQual","GarageCond","BsmtExposure",
              "Neighborhood","LandSlope","LotConfig","Utilities","LandContour","LotShape","Street","MSZoning",
              "SaleType","PavedDrive","GarageType","HouseStyle","Functional","BsmtFinType1","BsmtQual",
              "BsmtFinType2","Foundation","ExterCond","ExterQual","SaleCondition","HeatingQC","CentralAir",
              "Electrical","MasVnrType","Exterior2nd","Exterior1st","RoofMatl","RoofStyle","KitchenQual",
              "BsmtCond","BldgType","Heating","MSSubClass","OverallQual","OverallCond","BsmtFullBath",
              "BsmtHalfBath","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","Fireplaces","GarageCars",
              "MoSold","YrSold","TotRmsAbvGrd","YearRemodAdd","YearBuilt","GarageYrBlt"]
c_n_housing1 = [ele for ele in c_n_housing1 if ele not in unwanted_num]
print(c_n_housing1)
corrp = []

for i in c_n_housing1: 
    putt = X[i].corr(X['SalePrice'])
    
    if putt >= 0.5 or putt <-0.5:
        corrp = corrp + [i]
    else:
        continue
    print("The correlation between ", i ," and SalesPrice is = ", putt)
print(corrp)

for i in corrp:  # Loop over all columns except 'Location'
    sns.set()
    fig, ax = plt.subplots()
    fig.set_size_inches(30.5, 10.5)
    sns.regplot(x=i, y='SalePrice', data=X)  # column is chosen here
   

## The Cells below contain code to store the names of the categorical data to use for finding the P-value of the categorical columns. One column, GarageYrBlt, has missing data and needs to be filled first to be used, after which I calculate the P-value and and choose those that have a P-value of less than 0.001.

In [ ]:

pearson_housing = ["GarageFinish","Condition2","Condition1","GarageQual","GarageCond","BsmtExposure",
        "Neighborhood","LandSlope","LotConfig","Utilities","LandContour","LotShape","Street","MSZoning",
        "SaleType","PavedDrive","GarageType","HouseStyle","Functional","BsmtFinType1","BsmtQual",
        "BsmtFinType2","Foundation","ExterCond","ExterQual","SaleCondition","HeatingQC","CentralAir",
        "Electrical","MasVnrType","Exterior2nd","Exterior1st","RoofMatl","RoofStyle","KitchenQual",
        "BsmtCond","BldgType","Heating","MSSubClass","OverallQual","OverallCond","BsmtFullBath",
        "BsmtHalfBath","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","Fireplaces","GarageCars",
        "MoSold","YrSold","TotRmsAbvGrd","YearRemodAdd","YearBuilt","GarageYrBlt"]
X2 = X[["SalePrice","GarageFinish","Condition2","Condition1","GarageQual","GarageCond","BsmtExposure",
        "Neighborhood","LandSlope","LotConfig","Utilities","LandContour","LotShape","Street","MSZoning",
        "SaleType","PavedDrive","GarageType","HouseStyle","Functional","BsmtFinType1","BsmtQual",
        "BsmtFinType2","Foundation","ExterCond","ExterQual","SaleCondition","HeatingQC","CentralAir",
        "Electrical","MasVnrType","Exterior2nd","Exterior1st","RoofMatl","RoofStyle","KitchenQual",
        "BsmtCond","BldgType","Heating","MSSubClass","OverallQual","OverallCond","BsmtFullBath",
        "BsmtHalfBath","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","Fireplaces","GarageCars",
        "MoSold","YrSold","TotRmsAbvGrd","YearRemodAdd","YearBuilt","GarageYrBlt"]]
np.isnan(X2).any()

In [ ]:
X2['GarageYrBlt'].fillna(int((X2['GarageYrBlt'].mean())), inplace=True)

In [ ]:
np.isnan(X2).any()

In [ ]:
pcarl = []
for i in pearson_housing: 
   pearson_coef, p_value = stats.pearsonr(X2[i],X2['SalePrice'])
   if p_value <= 0.001:
        pcarl = pcarl + [i]
   else:
        continue
   print("The P-value of " ,i ," is P =", p_value) 

print(pcarl)

## The Cell below contains code to add polynomial features to try and augment the data to mitigate the variance and decrease the Mean Absolute Percentage Error (M.A.P.E). I will be trying to remove the polynomial data and leave the rest of the data to reduce variance.

In [ ]:
pr = pf(degree = 3, include_bias=False)
train_poly = pr.fit_transform(train[['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea']])
pr.get_feature_names_out(['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea'])
pd.DataFrame(train_poly,columns=['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea','TotalBsmtSF^2', 'TotalBsmtSF 1stFlrSF', 'TotalBsmtSF GrLivArea',
       'TotalBsmtSF GarageArea', '1stFlrSF^2', '1stFlrSF GrLivArea','1stFlrSF GarageArea', 'GrLivArea^2', 'GrLivArea GarageArea','GarageArea^2', 
       'TotalBsmtSF^3', 'TotalBsmtSF^2 1stFlrSF','TotalBsmtSF^2 GrLivArea', 'TotalBsmtSF^2 GarageArea','TotalBsmtSF 1stFlrSF^2', 
       'TotalBsmtSF 1stFlrSF GrLivArea','TotalBsmtSF 1stFlrSF GarageArea', 'TotalBsmtSF GrLivArea^2','TotalBsmtSF GrLivArea GarageArea', 
       'TotalBsmtSF GarageArea^2','1stFlrSF^3', '1stFlrSF^2 GrLivArea', '1stFlrSF^2 GarageArea','1stFlrSF GrLivArea^2', '1stFlrSF GrLivArea GarageArea',
       '1stFlrSF GarageArea^2', 'GrLivArea^3', 'GrLivArea^2 GarageArea','GrLivArea GarageArea^2', 'GarageArea^3'])

## The Cell below contains code to incorporate the newly made columns into the existing train dataset. It also begins by combining the features gotten from the preprocessing actvities before.

In [ ]:
train_clean = train[pcarl + corrp]
train_clean[["TotalBsmtSF", "1stFlrSF", "GrLivArea", "GarageArea","TotalBsmtSF^2", "TotalBsmtSF 1stFlrSF", "TotalBsmtSF GrLivArea",
       "TotalBsmtSF GarageArea", "1stFlrSF^2", "1stFlrSF GrLivArea","1stFlrSF GarageArea", "GrLivArea^2", "GrLivArea GarageArea","GarageArea^2", 
       "TotalBsmtSF^3", "TotalBsmtSF^2 1stFlrSF","TotalBsmtSF^2 GrLivArea", "TotalBsmtSF^2 GarageArea","TotalBsmtSF 1stFlrSF^2", 
       "TotalBsmtSF 1stFlrSF GrLivArea","TotalBsmtSF 1stFlrSF GarageArea", "TotalBsmtSF GrLivArea^2","TotalBsmtSF GrLivArea GarageArea", 
       "TotalBsmtSF GarageArea^2","1stFlrSF^3", "1stFlrSF^2 GrLivArea", "1stFlrSF^2 GarageArea","1stFlrSF GrLivArea^2", 
       "1stFlrSF GrLivArea GarageArea","1stFlrSF GarageArea^2", "GrLivArea^3", "GrLivArea^2 GarageArea","GrLivArea GarageArea^2", 
       "GarageArea^3"]] = train_poly
train_clean.columns.sort_values()

## The Cell below contains code to check the data types. 

In [ ]:
train_clean.dtypes

## The Cell below contains code to add polynomial features to match the test set with the training set.

In [ ]:
pr = pf(degree = 3, include_bias=False)
test_poly = pr.fit_transform(test[['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea']])
pr.get_feature_names_out(['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea'])
pd.DataFrame(test_poly,columns=['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea','TotalBsmtSF^2', 'TotalBsmtSF 1stFlrSF', 'TotalBsmtSF GrLivArea',
       'TotalBsmtSF GarageArea', '1stFlrSF^2', '1stFlrSF GrLivArea','1stFlrSF GarageArea', 'GrLivArea^2', 'GrLivArea GarageArea','GarageArea^2', 
       'TotalBsmtSF^3', 'TotalBsmtSF^2 1stFlrSF','TotalBsmtSF^2 GrLivArea', 'TotalBsmtSF^2 GarageArea','TotalBsmtSF 1stFlrSF^2', 
       'TotalBsmtSF 1stFlrSF GrLivArea','TotalBsmtSF 1stFlrSF GarageArea', 'TotalBsmtSF GrLivArea^2','TotalBsmtSF GrLivArea GarageArea', 
       'TotalBsmtSF GarageArea^2','1stFlrSF^3', '1stFlrSF^2 GrLivArea', '1stFlrSF^2 GarageArea','1stFlrSF GrLivArea^2', '1stFlrSF GrLivArea GarageArea',
       '1stFlrSF GarageArea^2', 'GrLivArea^3', 'GrLivArea^2 GarageArea','GrLivArea GarageArea^2', 'GarageArea^3'])

## The Cell below contains code to incorporate the newly made columns into the existing test dataset. It also begins by combining the features gotten from the preprocessing actvities before to match with the train dataset.

In [ ]:
test_clean = test[pcarl + corrp]
test_clean[["TotalBsmtSF", "1stFlrSF", "GrLivArea", "GarageArea","TotalBsmtSF^2", "TotalBsmtSF 1stFlrSF", "TotalBsmtSF GrLivArea",
       "TotalBsmtSF GarageArea", "1stFlrSF^2", "1stFlrSF GrLivArea","1stFlrSF GarageArea", "GrLivArea^2", "GrLivArea GarageArea","GarageArea^2", 
       "TotalBsmtSF^3", "TotalBsmtSF^2 1stFlrSF","TotalBsmtSF^2 GrLivArea", "TotalBsmtSF^2 GarageArea","TotalBsmtSF 1stFlrSF^2", 
       "TotalBsmtSF 1stFlrSF GrLivArea","TotalBsmtSF 1stFlrSF GarageArea", "TotalBsmtSF GrLivArea^2","TotalBsmtSF GrLivArea GarageArea", 
       "TotalBsmtSF GarageArea^2","1stFlrSF^3", "1stFlrSF^2 GrLivArea", "1stFlrSF^2 GarageArea","1stFlrSF GrLivArea^2", 
       "1stFlrSF GrLivArea GarageArea","1stFlrSF GarageArea^2", "GrLivArea^3", "GrLivArea^2 GarageArea","GrLivArea GarageArea^2", 
       "GarageArea^3"]] = test_poly
test_clean.columns.sort_values()

## The Cell below contains code to Divide the train dataset into x and y. Y contains the expected output and x contains the features used to get it.

In [ ]:
XX = train_clean
YY = train.SalePrice

## The Cell below contains code to fill all the missisng features in the column GarageYrBlt.

In [ ]:
XX.fillna(1, inplace=True)
XX["GarageYrBlt"] = XX["GarageYrBlt"].astype('int64')
XX["GarageYrBlt"]

## The Cell below contains code to define the categorical features.

In [ ]:
cat_features1 = pcarl

## The Cell below contains code to split the data into train and test data, define the sample space for the Optuna algorithm to tune the hyperparameters and define the output to be used to tune the hyperparameters. I use MAPE of the test data and the variance between the training set and the test set.

In [ ]:
params_list = []
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(XX,YY,test_size=0.25,random_state=42)


Train_set=Pool(X_train, Y_train,cat_features=cat_features1)

Eval_set=Pool(X_test, Y_test,cat_features=cat_features1)
def objective(trial):
    global params_list
    param = {
        #'task_type':"GPU",
        'iterations':5000,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.000001, 0.01),
        'use_best_model':True,
        #'od_type' : "Iter",
        #'od_wait' : 500,
        #'random_seed': 240,
        #"scale_pos_weight":trial.suggest_int("scale_pos_weight", 1, 10),
        "depth": trial.suggest_int("max_depth", 1, 10),
        "l2_leaf_reg": trial.suggest_loguniform("lambda",1,5),
          'eval_metric':trial.suggest_categorical("loss_function",['RMSE']),
         'one_hot_max_size':256
        }

    # Add a callback for pruning.
    model=CatBoostRegressor(**param)
    print(param)
    model.fit(Train_set,eval_set=Eval_set,plot=False,verbose=False)
    pred1 = model.predict(Pool(X_train,cat_features= cat_features1))
    pred2 = model.predict(Pool(X_test,cat_features= cat_features1))
    mape1 = (metrics.mean_absolute_percentage_error(pred1,Y_train))*100 
    mape2 = (metrics.mean_absolute_percentage_error(pred2,Y_test))*100
    variance = mape2-mape1
    print("The MAPE for train is", mape1)
    print("The MAPE for test is", mape2)
    print("The VARIANCE is ",variance)
   
    return mape2,variance

## The Cell below contains code to activate the Optuna Hyperparameter tuning algorithm. It also sets the number of trials that Optuna will run on the provided hyperparameter sample space above. And it also has code for the best trials to be outputted below the trials code.

In [ ]:
if __name__ == "__main__":
    study = optuna.create_study(directions=["minimize", "minimize"])
    study.optimize(objective, n_trials=1000)
    trial_with_lowest_error = max(study.best_trials, key=lambda t: t.values[1])
    print(f"Trial_with_Lowest_error: ")
    print(f"\tnumber: {trial_with_lowest_error.number}")
    print(f"\tparams: {trial_with_lowest_error.params}")
    print(f"\tvalues: {trial_with_lowest_error.values}")
    trial_with_lowest_variance = max(study.best_trials, key=lambda t: t.values[0])
    print(f"Trial_with_Lowest_variance: ")
    print(f"\tnumber: {trial_with_lowest_variance.number}")
    print(f"\tparams: {trial_with_lowest_variance.params}")
    print(f"\tvalues: {trial_with_lowest_variance.values}")
    

## The Cell below contains code to plot a new experimental feature from Optuna that Plots the two objectives I am using to tune the hyperparameters to find the best hypereparameters together with the importance of the different hyperparameters in the next two code cells for MAPE of the test set and the variance between the MAPE of the training set and the MAPE of the test set respectively. 

In [ ]:
optuna.visualization.plot_pareto_front(study, target_names=["mape2", "variance"])

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0], target_name="mape2")

In [ ]:
optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0], target_name="variance")

## The Cell below contains code to execute the selected hyperparameters in CatBoostRegressor Algorithm. As before it produces both the training and the test set MAPE to measure the variance.

In [ ]:
param={'one_hot_max_size':256,
       'iterations': 10000,
       'learning_rate': 0.0009402273445644647,
       'use_best_model': True, 
       #'od_type': 'Iter'
       #'od_wait': 10000, 
       'max_depth': 3,
       'l2_leaf_reg': 1.196467054178472,
       'loss_function':'RMSE'}
model=CatBoostRegressor(**param)
print(param)
model.fit(Train_set,eval_set=Eval_set,plot=False,verbose=False)
pred1 = model.predict(Pool(X_train,cat_features= cat_features1))
pred2 = model.predict(Pool(X_test,cat_features= cat_features1))
mape1 = (metrics.mean_absolute_percentage_error(pred1,Y_train))*100 
mape2 = (metrics.mean_absolute_percentage_error(pred2,Y_test))*100
print("The MAPE for train is", mape1)
print("The MAPE for test is", mape2)


# I will add the cell for outputting the submission file here after I am done with the trianing and I am sure that the model can properly generalize with the lowest error possible.